# Healthcare AI Assistant with Multimodal + Insurance Tool Support

This Jupyter Notebook demonstrates how to build an AI-powered **healthcare assistant** using OpenAI's GPT-4o model.  
The assistant is capable of:

- Responding to patient or provider questions about EHR, CPT/ICD codes, and health insurance claims
- Using simulated tools for **CPT/ICD-10 lookup** and **Medicare/Medicaid insurance coverage**
- Processing **images** (e.g., medical records, screenshots) via OpenAI's **multimodal** capabilities
- Deploying an interactive chatbot interface using **Gradio**

The assistant mimics a real-world scenario where AI augments healthcare workflows by answering coverage, diagnosis, or billing-related questions efficiently and accurately.


In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [2]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key loaded: {openai_api_key[:8]}******")
else:
    print("OpenAI API Key is not set.")

MODEL = "gpt-4o"
openai = OpenAI()


OpenAI API Key loaded: sk-orYYT******


In [3]:
# System message
system_message = (
    "You are a helpful and medically accurate AI assistant specialized in healthcare. "
    "You can analyze text, images (e.g., EHR screenshots or forms), and structured claims data. "
    "Answer questions about electronic health records (EHR), health insurance claims, CPT codes, coverage, and patient support. "
    "Be concise, professional, and cite medical guidelines if known. If you are unsure, say so clearly."
)

In [4]:
# CPT/ICD-10 lookup tool
def lookup_code(code):
    code = code.strip().upper()
    sample_codes = {
        "99213": "Office or other outpatient visit for the evaluation and management of an established patient",
        "E11.9": "Type 2 diabetes mellitus without complications",
        "Z00.00": "General adult medical examination without abnormal findings"
    }
    return sample_codes.get(code, "Code not found. Please verify the CPT or ICD-10 code.")

In [5]:
# Medicare/Medicaid coverage simulation tool
def check_insurance_coverage(query):
    coverage_info = {
        "diabetes screening": "Medicare Part B covers diabetes screening tests if you have risk factors such as high blood pressure, history of abnormal cholesterol, obesity, or a family history of diabetes. Up to 2 screenings per year are covered at no cost.",
        "colonoscopy": "Medicare Part B covers screening colonoscopies once every 24 months (if high risk) or every 10 years (if not high risk).",
        "flu shot": "Medicare covers one flu shot per season with no copay if the provider accepts assignment.",
        "cpt 99495": "CPT 99495 (transitional care management services) is usually covered by Medicare and many private insurers when conditions are met, including communication within 2 business days of discharge and a face-to-face visit within 14 days."
    }

    key = query.lower()
    for k in coverage_info:
        if k in key:
            return f"[Insurance Coverage Info]\n{k.title()}: {coverage_info[k]}"
    return "Coverage information not found. Please verify the procedure or service name."

In [6]:
# Chat function with tools
def chat_with_tool(prompt):
    prompt_upper = prompt.upper()
    if any(code in prompt_upper for code in ["99213", "E11.9", "Z00.00"]):
        parts = prompt_upper.split()
        found_codes = [code for code in parts if code in ["99213", "E11.9", "Z00.00"]]
        if found_codes:
            tool_result = "\n".join([f"{code}: {lookup_code(code)}" for code in found_codes])
            return f"[From CPT/ICD Lookup Tool]\n{tool_result}"

    coverage_response = check_insurance_coverage(prompt)
    if "not found" not in coverage_response:
        return coverage_response

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
    ]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=0.3
    )
    return response.choices[0].message.content


In [ ]:
# Multimodal image + text understanding
def analyze_image_and_text(image, question):
    base64_img = None
    if image:
        import base64
        from PIL import Image
        from io import BytesIO

        buffered = BytesIO()
        image.save(buffered, format="PNG")
        base64_img = base64.b64encode(buffered.getvalue()).decode()

    messages = [{"role": "system", "content": system_message}]
    if question:
        messages.append({"role": "user", "content": question})
    if base64_img:
        messages.append({"role": "user", "content": [{"type": "image_url", "image_url": {"url": f"data:image/png;base64,{base64_img}"}}]})

    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return response.choices[0].message.content

In [8]:
# Gradio Interface
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown("## 🏥 Healthcare AI Assistant with CPT/ICD Tool and Multimodal Support")

    with gr.Tab("Chat with Text"):
        text_input = gr.Textbox(label="Ask a question (EHR, claims, insurance, CPT codes)")
        text_output = gr.Textbox(label="Response")
        text_button = gr.Button("Submit")
        text_button.click(fn=chat_with_tool, inputs=text_input, outputs=text_output)

    with gr.Tab("Chat with Image + Question"):
        img_input = gr.Image(label="Upload medical image or screenshot", type="pil")
        img_question = gr.Textbox(label="Ask a question about the image (optional)")
        img_output = gr.Textbox(label="Response")
        gr.Button("Analyze").click(fn=analyze_image_and_text, inputs=[img_input, img_question], outputs=img_output)

demo.launch()


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


---

## Summary

In this project, we've created a robust healthcare AI assistant that integrates:
-  **Text-based medical Q&A**
-  **Tool augmentation** with CPT/ICD-10 and insurance coverage lookups
-  **Image understanding** with multimodal GPT-4o
-  **Interactive chatbot** powered by Gradio

Applications: we can expand this assistant by connecting to:
- Real-time ICD/CPT APIs (e.g., CMS, AMA)
- Electronic Health Record (EHR) systems via FHIR/HL7
- External payer databases (for up-to-date coverage info)

This notebook provides a strong foundation for building practical, AI-powered assistants for healthcare, claims analysis, and patient support.

Feel free to modify, deploy, or extend it for your own use cases!
